In [1]:
import pandas as pd
metadf = pd.read_csv("drive/Shareddrives/810/metadata.csv")


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
!pip3 install pyspark

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 49.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=3f466d65faac0d64ba586b69668650367acff3cb5c218cf3e616e5fe741f31fe
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession

In [5]:
MAX_MEMORY = "20g"
spark = SparkSession \
    .builder \
    .appName("EcommerceAnalysis") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .config("spark.ui.port", "4040") \
    .master("local[*]") \
    .getOrCreate()
spark

In [7]:
df = spark \
    .read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(f"/content/drive/Shareddrives/810/metadata.csv")

In [8]:
df.write \
    .mode("overwrite") \
    .option("compression", "gzip")\
    .parquet("kag_pa")

In [9]:
df = spark.read.parquet("kag_pa").persist()

In [10]:
df.show()

+--------+----+--------+--------------------+--------------------+-----+---------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|cord_uid| sha|source_x|               title|                 doi|pmcid|pubmed_id|license|            abstract|        publish_time|             authors|             journal|              mag_id|    who_covidence_id|            arxiv_id|      pdf_json_files|      pmc_json_files|                 url|               s2_id|
+--------+----+--------+--------------------+--------------------+-----+---------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|zjsgzqgz|null| Medline|Practical 

In [11]:
df.count()

902589

In [13]:
_df = df.select("pmcid").where("pmcid like '%PMC%' and pmc_json_files != 'null'")
_df.toPandas().to_csv('pmcidlist.csv')


In [12]:
df.registerTempTable("kag")

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


In [14]:
spark.sql("select * from kag limit 5").show()

+--------+----+--------+--------------------+--------------------+-----+---------+-------+--------------------+------------+--------------------+--------------------+------+----------------+--------+--------------+--------------+--------------------+---------+
|cord_uid| sha|source_x|               title|                 doi|pmcid|pubmed_id|license|            abstract|publish_time|             authors|             journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|                 url|    s2_id|
+--------+----+--------+--------------------+--------------------+-----+---------+-------+--------------------+------------+--------------------+--------------------+------+----------------+--------+--------------+--------------+--------------------+---------+
|zjsgzqgz|null| Medline|Practical home-ba...|10.23736/s0022-47...| null| 33092335|    unk|                null|  2020-10-22|Zhang, Shiyan; Yu...|The Journal of sp...|  null|            null|    null|          null|   

In [15]:
spark.sql("""


select count(pmcid) from kag where pmcid like 'PMC%'

""").show()

+------------+
|count(pmcid)|
+------------+
|      325252|
+------------+



In [ ]:
spark.sql("""


select count(pmcid) from kag where pmcid like 'PMC%' and publish_time>'2019-12-01'

""").show()

+------------+
|count(pmcid)|
+------------+
|      273983|
+------------+



In [16]:
spark.sql("""


select pmcid, publish_time, title, authors, abstract from kag where pmcid like 'PMC%' and publish_time>='2019-12-01'
order by publish_time

""").show()

+----------+------------+--------------------+--------------------+--------------------+
|     pmcid|publish_time|               title|             authors|            abstract|
+----------+------------+--------------------+--------------------+--------------------+
|PMC7327228|  2019-12-01|Implementing the ...|Seal, Hayley E.; ...|Children with con...|
|PMC8422197|  2019-12-01|Message from Depu...|    Leung, Gabriel M|                null|
|PMC8422199|  2019-12-01|One Hundred Years...|Zhang, Ran; Dong,...|Almost 100 years ...|
|PMC7122531|  2019-12-01|Complication from...|      Han, Duck Jong|Sensitization to ...|
|PMC7247748|  2019-12-01|Immunogenicity of...|Thibau, Arno; Dic...|The current probl...|
|PMC6874263|  2019-12-01|MERS-CoV in Camel...|Farag, Elmoubashe...|We tested samples...|
|PMC7358814|  2019-12-01|2019 NOVEL CORONA...|       Fagbule, O.F.|                null|
|PMC6863388|  2019-12-01|Characterisation ...|Lawson, J.S.; Sym...|The Crandell-Rees...|
|PMC6962603|  2019-12

Import the title SIA result

In [17]:
result = spark \
    .read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(f"/content/drive/Shareddrives/810/title_result.csv")


result.write \
    .mode("overwrite") \
    .option("compression", "gzip")\
    .parquet("kag_lb")

result = spark.read.parquet("kag_lb").persist()

result.registerTempTable("result")

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


In [18]:
spark.sql("""


select * from result 

""").show()

+---+-----+-----+-----+--------+--------------------+----------+-----+
|_c0|  neg|  neu|  pos|compound|               title|  paper_id|label|
+---+-----+-----+-----+--------+--------------------+----------+-----+
|  0|  0.0|  1.0|  0.0|     0.0|Timing of surgery...|PMC8206995|  0.0|
|  1|  0.0|  1.0|  0.0|     0.0|     Poster Sessions|PMC7111423|  0.0|
|  2|  0.0|  1.0|  0.0|     0.0|Cardiovascular Ac...|PMC7122603|  0.0|
|  3|  0.0|  1.0|  0.0|     0.0|             Posters|PMC7162159|  0.0|
|  4|  0.0|  1.0|  0.0|     0.0|             Posters|PMC7130089|  0.0|
|  5|  0.0|  1.0|  0.0|     0.0|Foot & mouth disease|PMC7111224|  0.0|
|  6|  0.0|  1.0|  0.0|     0.0|SARS‐CoV‐2 infect...|PMC8652887|  0.0|
|  7|  0.0|0.697|0.303|  0.7269|SARS-CoV-2 vaccin...|PMC7995808|  1.0|
|  8|  0.0|  1.0|  0.0|     0.0|Fusarium: more th...|PMC8379525|  0.0|
|  9|  0.0|  1.0|  0.0|     0.0|Infektionsschutz ...|PMC7152143|  0.0|
| 10|  0.0|  1.0|  0.0|     0.0|Fungal diversity ...|PMC8648402|  0.0|
| 11|0

In [19]:
spark.sql("""
with merged_df as (
    select pmcid, publish_time, title as md_title, authors, abstract
    from kag 
    where pmcid like 'PMC%' and publish_time>='2019-12-01'
    order by kag.publish_time
  )

select pmcid, publish_time, md_title, authors, abstract, neg, neu, pos, compound, label
from merged_df
INNER JOIN result ON merged_df.pmcid=result.paper_id;

""").show()



+----------+------------+--------------------+--------------------+--------------------+-----+-----+-----+--------+-----+
|     pmcid|publish_time|            md_title|             authors|            abstract|  neg|  neu|  pos|compound|label|
+----------+------------+--------------------+--------------------+--------------------+-----+-----+-----+--------+-----+
|PMC6874235|  2019-12-01|Middle East Respi...|Zheng, Jian; Hass...|A high percentage...|  0.0|  1.0|  0.0|     0.0|  0.0|
|PMC6890879|  2019-12-03|Evaluation of ant...|Peng, Ju-Yi; Horn...|Bacillus lichenif...|  0.0|  1.0|  0.0|     0.0|  0.0|
|PMC6898929|  2019-12-06|Risk factors for ...|Gupta, Ena; Hosse...|BACKGROUND: Clini...|0.244|0.756|  0.0| -0.6705| -1.0|
|PMC6899506|  2019-12-01|A systematic revi...|Dighe, Amy; Jomba...|Human infection w...|  0.0|  1.0|  0.0|     0.0|  0.0|
|PMC6901795|  2019-12-03|Zebrafish TRIM25 ...|Jin, Yilin; Jia, ...|RIG-I-like recept...|  0.0|0.796|0.204|  0.5574|  1.0|
|PMC6902615|  2019-12-09

In [20]:
spark.sql("""
with merged_df as (
    select pmcid, publish_time, title as md_title, authors, abstract
    from kag 
    where pmcid like 'PMC%' and publish_time>='2019-12-01'
    order by kag.publish_time),

 df as (
  select pmcid, publish_time, md_title, authors, abstract, neg, neu, pos, compound, label
from merged_df
INNER JOIN result ON merged_df.pmcid=result.paper_id)

select  pmcid, EXTRACT(YEAR FROM publish_time) as publish_year, md_title, authors, abstract, neg, neu, pos, compound, label
from df

""").show()


+----------+------------+--------------------+--------------------+--------------------+-----+-----+-----+--------+-----+
|     pmcid|publish_year|            md_title|             authors|            abstract|  neg|  neu|  pos|compound|label|
+----------+------------+--------------------+--------------------+--------------------+-----+-----+-----+--------+-----+
|PMC6874235|        2019|Middle East Respi...|Zheng, Jian; Hass...|A high percentage...|  0.0|  1.0|  0.0|     0.0|  0.0|
|PMC6890879|        2019|Evaluation of ant...|Peng, Ju-Yi; Horn...|Bacillus lichenif...|  0.0|  1.0|  0.0|     0.0|  0.0|
|PMC6898929|        2019|Risk factors for ...|Gupta, Ena; Hosse...|BACKGROUND: Clini...|0.244|0.756|  0.0| -0.6705| -1.0|
|PMC6899506|        2019|A systematic revi...|Dighe, Amy; Jomba...|Human infection w...|  0.0|  1.0|  0.0|     0.0|  0.0|
|PMC6901795|        2019|Zebrafish TRIM25 ...|Jin, Yilin; Jia, ...|RIG-I-like recept...|  0.0|0.796|0.204|  0.5574|  1.0|
|PMC6902615|        2019

In [21]:
spark.sql("""
with merged_df as (
    select pmcid, publish_time, title as md_title, authors, abstract
    from kag 
    where pmcid like 'PMC%' and publish_time>='2020-01-01' and publish_time<='2023-01-01'
    order by kag.publish_time),

 df as (
  select pmcid, publish_time, md_title, authors, abstract, neg, neu, pos, compound, label
from merged_df
INNER JOIN result ON merged_df.pmcid=result.paper_id)

select EXTRACT(YEAR FROM publish_time) as publish_year,avg(compound)
from df
group by publish_year

order by publish_year


""").show()

+------------+--------------------+
|publish_year|       avg(compound)|
+------------+--------------------+
|        2020|-0.00853388939756...|
|        2021|-0.00409004964988...|
|        2022|0.004802362984040585|
+------------+--------------------+



In [23]:
spark.stop()